In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [62]:
secrets_file = open("secrets.txt","r")
#opening just for reading "r"

In [63]:
string = secrets_file.read()
#string

In [64]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [ ]:
#secrets_dict

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['Client ID'],
                                                           client_secret=secrets_dict['Client Secret']))

In [9]:
playlist = sp.user_playlist_tracks("spotify", "6yPiKpy7evrwvZodByKvM9")

In [10]:
#playlist

In [11]:
playlist.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [12]:
playlist["items"][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [13]:
playlist["items"][0]["track"]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/50Lr1puweM1hFsF1LpIZLM'},
    'href': 'https://api.spotify.com/v1/artists/50Lr1puweM1hFsF1LpIZLM',
    'id': '50Lr1puweM1hFsF1LpIZLM',
    'name': 'WhoMadeWho',
    'type': 'artist',
    'uri': 'spotify:artist:50Lr1puweM1hFsF1LpIZLM'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/1cS5EoXVqz7aON00AkVBBA'},
    'href': 'https://api.spotify.com/v1/artists/1cS5EoXVqz7aON00AkVBBA',
    'id': '1cS5EoXVqz7aON00AkVBBA',
    'name': 'DAVID AUGUST',
    'type': 'artist',
    'uri': 'spotify:artist:1cS5EoXVqz7aON00AkVBBA'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/2GvwZbDjH1DbQpodGKENDw'},
    'href': 'https://api.spotify.com/v1/artists/2GvwZbDjH1DbQpodGKENDw',
    'id': '2GvwZbDjH1DbQpodGKENDw',
    'name': 'Roman Flügel',
    'type': 'artist',
    'uri': 'spotify:artist:2GvwZbDjH1DbQpodGKENDw'}],
  'available_markets': [],
  'external_urls'

In [14]:
playlist["items"][0]['track']['artists'][0]['name']

'Nosaj Thing'

In [15]:
playlist["items"][0]["track"]["name"]

'2K'

In [16]:
playlist["total"]

10000

In [17]:
playlist["next"]

'https://api.spotify.com/v1/playlists/6yPiKpy7evrwvZodByKvM9/tracks?offset=100&limit=100&additional_types=track'

In [18]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [19]:
all_tracks = get_playlist_tracks("6yPiKpy7evrwvZodByKvM9")
len(all_tracks)

10000

In [20]:
import pandas as pd
from pandas import json_normalize
pd.set_option("display.max_columns", 0)

In [21]:
tracks2 = json_normalize(all_tracks)

In [22]:
tracks2.columns

Index(['added_at', 'is_local', 'primary_color',
       'added_by.external_urls.spotify', 'added_by.href', 'added_by.id',
       'added_by.type', 'added_by.uri', 'track.album.album_type',
       'track.album.artists', 'track.album.available_markets',
       'track.album.external_urls.spotify', 'track.album.href',
       'track.album.id', 'track.album.images', 'track.album.name',
       'track.album.release_date', 'track.album.release_date_precision',
       'track.album.total_tracks', 'track.album.type', 'track.album.uri',
       'track.artists', 'track.available_markets', 'track.disc_number',
       'track.duration_ms', 'track.episode', 'track.explicit',
       'track.external_ids.isrc', 'track.external_urls.spotify', 'track.href',
       'track.id', 'track.is_local', 'track.name', 'track.popularity',
       'track.preview_url', 'track.track', 'track.track_number', 'track.type',
       'track.uri', 'video_thumbnail.url'],
      dtype='object')

In [23]:
len(tracks2)

10000

In [24]:
artists_df = pd.DataFrame(columns=['track.name', 'track.id', 'track.popularity'])
for i in tracks2.index:
    artists_for_song = json_normalize(tracks2.iloc[i]['track.artists'])
    artists_for_song['song_id']    = tracks2.iloc[i]['track.id']         # we want to keep song_id, it is the sae for all artists
    artists_for_song['song_name']  = tracks2.iloc[i]['track.name']       # we want to keep song_name, it is the sae for all artists
    artists_for_song['popularity'] = tracks2.iloc[i]['track.popularity']
    #artists_for_song['artist'] = tracks2.iloc[i]['name']
    artists_df = pd.concat([artists_df, artists_for_song], axis=0)

In [25]:
artists_df.head(20)

,track.name,track.id,track.popularity,href,id,name,type,uri,external_urls.spotify,song_id,song_name,popularity
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/0IVapwlnM3d...,0IVapwlnM3dEOiMsHXsghT,Nosaj Thing,artist,spotify:artist:0IVapwlnM3dEOiMsHXsghT,https://open.spotify.com/artist/0IVapwlnM3dEOi...,33xMbeHzmWd6Od0BmLZEUs,2K,0.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/22WZ7M8sxp5...,22WZ7M8sxp5THdruNY3gXt,The Doors,artist,spotify:artist:22WZ7M8sxp5THdruNY3gXt,https://open.spotify.com/artist/22WZ7M8sxp5THd...,3UnyplmZaq547hwsfOR5yy,4 Billion Souls,26.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/4Z8W4fKeB5Y...,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,artist,spotify:artist:4Z8W4fKeB5YxbusRsdQVPb,https://open.spotify.com/artist/4Z8W4fKeB5Yxbu...,1w8QCSDH4QobcQeT4uMKLm,4 Minute Warning,0.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/0UK6JkgUMa2...,0UK6JkgUMa28b4t8eCtg6P,Vitas,artist,spotify:artist:0UK6JkgUMa28b4t8eCtg6P,https://open.spotify.com/artist/0UK6JkgUMa28b4...,7J9mBHG4J2eIfDAv5BehKA,7 Element,0.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/4KWTAlx2Rvb...,4KWTAlx2RvbpseOGMEmROg,R.E.M.,artist,spotify:artist:4KWTAlx2RvbpseOGMEmROg,https://open.spotify.com/artist/4KWTAlx2Rvbpse...,1VZedwJj1gyi88WFRhfThb,#9 Dream,6.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/2jzc5TC5TVF...,2jzc5TC5TVFLXQlBNiIUzE,a-ha,artist,spotify:artist:2jzc5TC5TVFLXQlBNiIUzE,https://open.spotify.com/artist/2jzc5TC5TVFLXQ...,3DSmOF7ETJHAQEqqgoSJao,#9 Dream,12.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/7mnBLXK823v...,7mnBLXK823vNxN3UWB7Gfz,The Black Keys,artist,spotify:artist:7mnBLXK823vNxN3UWB7Gfz,https://open.spotify.com/artist/7mnBLXK823vNxN...,5HPnumlogzZtpLEaORZjz1,10 Lovers,53.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/1rAv1GhTQ2r...,1rAv1GhTQ2rmG94p9lU3rB,Julian Casablancas,artist,spotify:artist:1rAv1GhTQ2rmG94p9lU3rB,https://open.spotify.com/artist/1rAv1GhTQ2rmG9...,6i99ThziImjuFABb8rUQvy,11th Dimension,56.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/0epOFNiUfyO...,0epOFNiUfyON9EYx7Tpr6V,The Strokes,artist,spotify:artist:0epOFNiUfyON9EYx7Tpr6V,https://open.spotify.com/artist/0epOFNiUfyON9E...,0nkLI0pdyTRpq7BsTFBufZ,12:51,0.0
0,NaN,NaN,NaN,https://api.spotify.com/v1/artists/0epOFNiUfyO...,0epOFNiUfyON9EYx7Tpr6V,The Strokes,artist,spotify:artist:0epOFNiUfyON9EYx7Tpr6V,https://open.spotify.com/artist/0epOFNiUfyON9E...,6Yu1OL8I0D4vjOzYdsXYGK,12:51,71.0


In [26]:
artists_df = artists_df.drop(['track.name', 'track.id', 'track.popularity'], axis=1)
artists_df.head(20)

,href,id,name,type,uri,external_urls.spotify,song_id,song_name,popularity
0,https://api.spotify.com/v1/artists/0IVapwlnM3d...,0IVapwlnM3dEOiMsHXsghT,Nosaj Thing,artist,spotify:artist:0IVapwlnM3dEOiMsHXsghT,https://open.spotify.com/artist/0IVapwlnM3dEOi...,33xMbeHzmWd6Od0BmLZEUs,2K,0.0
0,https://api.spotify.com/v1/artists/22WZ7M8sxp5...,22WZ7M8sxp5THdruNY3gXt,The Doors,artist,spotify:artist:22WZ7M8sxp5THdruNY3gXt,https://open.spotify.com/artist/22WZ7M8sxp5THd...,3UnyplmZaq547hwsfOR5yy,4 Billion Souls,26.0
0,https://api.spotify.com/v1/artists/4Z8W4fKeB5Y...,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,artist,spotify:artist:4Z8W4fKeB5YxbusRsdQVPb,https://open.spotify.com/artist/4Z8W4fKeB5Yxbu...,1w8QCSDH4QobcQeT4uMKLm,4 Minute Warning,0.0
0,https://api.spotify.com/v1/artists/0UK6JkgUMa2...,0UK6JkgUMa28b4t8eCtg6P,Vitas,artist,spotify:artist:0UK6JkgUMa28b4t8eCtg6P,https://open.spotify.com/artist/0UK6JkgUMa28b4...,7J9mBHG4J2eIfDAv5BehKA,7 Element,0.0
0,https://api.spotify.com/v1/artists/4KWTAlx2Rvb...,4KWTAlx2RvbpseOGMEmROg,R.E.M.,artist,spotify:artist:4KWTAlx2RvbpseOGMEmROg,https://open.spotify.com/artist/4KWTAlx2Rvbpse...,1VZedwJj1gyi88WFRhfThb,#9 Dream,6.0
0,https://api.spotify.com/v1/artists/2jzc5TC5TVF...,2jzc5TC5TVFLXQlBNiIUzE,a-ha,artist,spotify:artist:2jzc5TC5TVFLXQlBNiIUzE,https://open.spotify.com/artist/2jzc5TC5TVFLXQ...,3DSmOF7ETJHAQEqqgoSJao,#9 Dream,12.0
0,https://api.spotify.com/v1/artists/7mnBLXK823v...,7mnBLXK823vNxN3UWB7Gfz,The Black Keys,artist,spotify:artist:7mnBLXK823vNxN3UWB7Gfz,https://open.spotify.com/artist/7mnBLXK823vNxN...,5HPnumlogzZtpLEaORZjz1,10 Lovers,53.0
0,https://api.spotify.com/v1/artists/1rAv1GhTQ2r...,1rAv1GhTQ2rmG94p9lU3rB,Julian Casablancas,artist,spotify:artist:1rAv1GhTQ2rmG94p9lU3rB,https://open.spotify.com/artist/1rAv1GhTQ2rmG9...,6i99ThziImjuFABb8rUQvy,11th Dimension,56.0
0,https://api.spotify.com/v1/artists/0epOFNiUfyO...,0epOFNiUfyON9EYx7Tpr6V,The Strokes,artist,spotify:artist:0epOFNiUfyON9EYx7Tpr6V,https://open.spotify.com/artist/0epOFNiUfyON9E...,0nkLI0pdyTRpq7BsTFBufZ,12:51,0.0
0,https://api.spotify.com/v1/artists/0epOFNiUfyO...,0epOFNiUfyON9EYx7Tpr6V,The Strokes,artist,spotify:artist:0epOFNiUfyON9EYx7Tpr6V,https://open.spotify.com/artist/0epOFNiUfyON9E...,6Yu1OL8I0D4vjOzYdsXYGK,12:51,71.0


In [27]:
artists_df = artists_df.drop(['href', 'id', 'type','uri','external_urls.spotify',], axis=1)
artists_df.head(20)

,name,song_id,song_name,popularity
0,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs,2K,0.0
0,The Doors,3UnyplmZaq547hwsfOR5yy,4 Billion Souls,26.0
0,Radiohead,1w8QCSDH4QobcQeT4uMKLm,4 Minute Warning,0.0
0,Vitas,7J9mBHG4J2eIfDAv5BehKA,7 Element,0.0
0,R.E.M.,1VZedwJj1gyi88WFRhfThb,#9 Dream,6.0
0,a-ha,3DSmOF7ETJHAQEqqgoSJao,#9 Dream,12.0
0,The Black Keys,5HPnumlogzZtpLEaORZjz1,10 Lovers,53.0
0,Julian Casablancas,6i99ThziImjuFABb8rUQvy,11th Dimension,56.0
0,The Strokes,0nkLI0pdyTRpq7BsTFBufZ,12:51,0.0
0,The Strokes,6Yu1OL8I0D4vjOzYdsXYGK,12:51,71.0


In [28]:
artists_df['song_id'].isna().sum()

82

In [29]:
artists_df.dropna(inplace=True)

In [30]:
artists_df['song_id'].isna().sum()

0

In [31]:
chunks = [(i, i+100) for i in range(0, len(artists_df), 100)]
chunks
audio_features_list = []
for chunk in chunks:
    id_list100 = artists_df['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

11771

In [32]:
audio_features_df = json_normalize(audio_features_list)

In [33]:
audio_features_df.drop_duplicates(inplace=True)

In [34]:
df_w_audio_ft = pd.merge(left=artists_df,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft

,name,song_id,song_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs,2K,0.0,0.310,0.445,7,-13.355,0,0.0863,0.09400,0.067800,0.1130,0.122,95.360,audio_features,33xMbeHzmWd6Od0BmLZEUs,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560,3
1,The Doors,3UnyplmZaq547hwsfOR5yy,4 Billion Souls,26.0,0.419,0.565,5,-11.565,1,0.0347,0.13700,0.337000,0.1280,0.648,151.277,audio_features,3UnyplmZaq547hwsfOR5yy,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707,4
2,Radiohead,1w8QCSDH4QobcQeT4uMKLm,4 Minute Warning,0.0,0.354,0.302,9,-13.078,1,0.0326,0.59000,0.070900,0.1110,0.223,123.753,audio_features,1w8QCSDH4QobcQeT4uMKLm,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285,4
3,Vitas,7J9mBHG4J2eIfDAv5BehKA,7 Element,0.0,0.727,0.785,5,-6.707,0,0.0603,0.32500,0.126000,0.3100,0.960,129.649,audio_features,7J9mBHG4J2eIfDAv5BehKA,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940,4
4,R.E.M.,1VZedwJj1gyi88WFRhfThb,#9 Dream,6.0,0.571,0.724,0,-5.967,1,0.0260,0.02310,0.003110,0.0919,0.385,116.755,audio_features,1VZedwJj1gyi88WFRhfThb,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11766,The Clash,5jzma6gCzYtKB1DbEwFZKH,London Calling - Remastered,75.0,0.651,0.801,0,-7.340,1,0.0514,0.12300,0.000000,0.0825,0.776,133.769,audio_features,5jzma6gCzYtKB1DbEwFZKH,spotify:track:5jzma6gCzYtKB1DbEwFZKH,https://api.spotify.com/v1/tracks/5jzma6gCzYtK...,https://api.spotify.com/v1/audio-analysis/5jzm...,200480,4
11767,War,2fmMPJb5EzZCx8BcNJvVk4,Low Rider,0.0,0.811,0.647,0,-10.989,1,0.0498,0.08230,0.681000,0.0572,0.990,139.787,audio_features,2fmMPJb5EzZCx8BcNJvVk4,spotify:track:2fmMPJb5EzZCx8BcNJvVk4,https://api.spotify.com/v1/tracks/2fmMPJb5EzZC...,https://api.spotify.com/v1/audio-analysis/2fmM...,191560,4
11768,Moby,60rIdEPDrzyLiLC0icp3xz,Flower,0.0,0.686,0.610,7,-5.902,1,0.0262,0.19000,0.000931,0.0710,0.766,80.567,audio_features,60rIdEPDrzyLiLC0icp3xz,spotify:track:60rIdEPDrzyLiLC0icp3xz,https://api.spotify.com/v1/tracks/60rIdEPDrzyL...,https://api.spotify.com/v1/audio-analysis/60rI...,206293,4
11769,The Cat Empire,0sEm1ld0V8YTCPcjPVfIsc,Brighter Than Gold,47.0,0.711,0.718,6,-5.739,1,0.0380,0.00824,0.002080,0.0732,0.688,117.071,audio_features,0sEm1ld0V8YTCPcjPVfIsc,spotify:track:0sEm1ld0V8YTCPcjPVfIsc,https://api.spotify.com/v1/tracks/0sEm1ld0V8YT...,https://api.spotify.com/v1/audio-analysis/0sEm...,200293,4


In [35]:
df_w_audio_ft['song_id'].isna().sum()

0

# next playlist

In [36]:
playlist2 = sp.user_playlist_tracks("spotify", "4Dg0J0ICj9kKTGDyFu0Cv4")

In [37]:
playlist2["total"]

1421

In [38]:
playlist2["next"]

'https://api.spotify.com/v1/playlists/4Dg0J0ICj9kKTGDyFu0Cv4/tracks?offset=100&limit=100&additional_types=track'

In [39]:
all_tracks2 = get_playlist_tracks("4Dg0J0ICj9kKTGDyFu0Cv4")
len(all_tracks2)

1421

In [40]:
tracks3 = json_normalize(all_tracks2)

In [41]:
tracks3.columns

Index(['added_at', 'is_local', 'primary_color',
       'added_by.external_urls.spotify', 'added_by.href', 'added_by.id',
       'added_by.type', 'added_by.uri', 'track.album.album_type',
       'track.album.artists', 'track.album.available_markets',
       'track.album.external_urls.spotify', 'track.album.href',
       'track.album.id', 'track.album.images', 'track.album.name',
       'track.album.release_date', 'track.album.release_date_precision',
       'track.album.total_tracks', 'track.album.type', 'track.album.uri',
       'track.artists', 'track.available_markets', 'track.disc_number',
       'track.duration_ms', 'track.episode', 'track.explicit',
       'track.external_ids.isrc', 'track.external_urls.spotify', 'track.href',
       'track.id', 'track.is_local', 'track.name', 'track.popularity',
       'track.preview_url', 'track.track', 'track.track_number', 'track.type',
       'track.uri', 'video_thumbnail.url'],
      dtype='object')

In [42]:
len(tracks3)

1421

In [43]:
artists_df2 = pd.DataFrame(columns=['track.name'])
for i in tracks3.index:
    artists_for_song1 = json_normalize(tracks3.iloc[i]['track.artists'])
    artists_for_song1['song_id']    = tracks3.iloc[i]['track.id']         # we want to keep song_id, it is the sae for all artists
    artists_for_song1['song_name']  = tracks3.iloc[i]['track.name']       # we want to keep song_name, it is the sae for all artists
    artists_for_song1['popularity'] = tracks3.iloc[i]['track.popularity'] # same for popularity   
    artists_df2 = pd.concat([artists_df2, artists_for_song1], axis=0)

In [44]:
artists_df2.head(20)

,track.name,href,id,name,type,uri,external_urls.spotify,song_id,song_name,popularity
0,NaN,https://api.spotify.com/v1/artists/4s3lxX76Lwx...,4s3lxX76LwxzMdQEAFYdzv,Tropa do Bruxo,artist,spotify:artist:4s3lxX76LwxzMdQEAFYdzv,https://open.spotify.com/artist/4s3lxX76LwxzMd...,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
1,NaN,https://api.spotify.com/v1/artists/0NB5wv4kn6A...,0NB5wv4kn6A919CLHUKRmk,DJ Ws da Igrejinha,artist,spotify:artist:0NB5wv4kn6A919CLHUKRmk,https://open.spotify.com/artist/0NB5wv4kn6A919...,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
2,NaN,https://api.spotify.com/v1/artists/2qATSoNFfzM...,2qATSoNFfzMaTEZFfJpG4c,SMU,artist,spotify:artist:2qATSoNFfzMaTEZFfJpG4c,https://open.spotify.com/artist/2qATSoNFfzMaTE...,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
3,NaN,https://api.spotify.com/v1/artists/4ACGdFl6mPI...,4ACGdFl6mPI3lntaYef1gc,Triz,artist,spotify:artist:4ACGdFl6mPI3lntaYef1gc,https://open.spotify.com/artist/4ACGdFl6mPI3ln...,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
4,NaN,https://api.spotify.com/v1/artists/3qjoOrsHcxh...,3qjoOrsHcxhgKyBsh4ZOEn,Mc Menor Thalis,artist,spotify:artist:3qjoOrsHcxhgKyBsh4ZOEn,https://open.spotify.com/artist/3qjoOrsHcxhgKy...,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
0,NaN,https://api.spotify.com/v1/artists/7Eu1txygG6n...,7Eu1txygG6nJttLHbZdQOh,Four Tet,artist,spotify:artist:7Eu1txygG6nJttLHbZdQOh,https://open.spotify.com/artist/7Eu1txygG6nJtt...,6fUo4GlD0FbvXNTylA4o68,Loved,62.0
0,NaN,https://api.spotify.com/v1/artists/0aIpJqqTLf6...,0aIpJqqTLf683ojWREc5lg,Joy Orbison,artist,spotify:artist:0aIpJqqTLf683ojWREc5lg,https://open.spotify.com/artist/0aIpJqqTLf683o...,4Wrd7TpAgRmlUtcbpJCW3T,flight fm,57.0
0,NaN,https://api.spotify.com/v1/artists/3TsEEdpuuCN...,3TsEEdpuuCN1G0dPxV4uOA,Koreless,artist,spotify:artist:3TsEEdpuuCN1G0dPxV4uOA,https://open.spotify.com/artist/3TsEEdpuuCN1G0...,4ekLUDFsOmaw90lAnRpaPJ,Seven (Drumhell),31.0
0,NaN,https://api.spotify.com/v1/artists/08Z0yhWGksN...,08Z0yhWGksNk3wceqlCeGE,Pangaea,artist,spotify:artist:08Z0yhWGksNk3wceqlCeGE,https://open.spotify.com/artist/08Z0yhWGksNk3w...,7yH6dUvFBeqRAb54z7z7GT,Squid,28.0
0,NaN,https://api.spotify.com/v1/artists/7Gg3X2b5ljr...,7Gg3X2b5ljrhVGXDAwcrq3,Cesco,artist,spotify:artist:7Gg3X2b5ljrhVGXDAwcrq3,https://open.spotify.com/artist/7Gg3X2b5ljrhVG...,0xHtnko6St2bKkMGgrtTO1,Swing King,51.0


In [45]:
artists_df2 = artists_df2.drop(['href','track.name', 'id', 'type','uri','external_urls.spotify'], axis=1)
artists_df2.head(20)

,name,song_id,song_name,popularity
0,Tropa do Bruxo,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
1,DJ Ws da Igrejinha,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
2,SMU,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
3,Triz,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
4,Mc Menor Thalis,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0
0,Four Tet,6fUo4GlD0FbvXNTylA4o68,Loved,62.0
0,Joy Orbison,4Wrd7TpAgRmlUtcbpJCW3T,flight fm,57.0
0,Koreless,4ekLUDFsOmaw90lAnRpaPJ,Seven (Drumhell),31.0
0,Pangaea,7yH6dUvFBeqRAb54z7z7GT,Squid,28.0
0,Cesco,0xHtnko6St2bKkMGgrtTO1,Swing King,51.0


In [46]:
chunks2 = [(i, i+100) for i in range(0, len(artists_df2), 100)]
chunks2
audio_features_list2 = []
for chunk in chunks:
    id_list1002 = artists_df2['song_id'][chunk[0]:chunk[1]]
    audio_features_list2 = audio_features_list2 + sp.audio_features(id_list1002)
    sleep(randint(1,3000)/1000)
len(audio_features_list2)

1756

In [47]:
audio_features_df2 = json_normalize(audio_features_list2)

In [48]:
audio_features_df2.drop_duplicates(inplace=True)

In [49]:
df_w_audio_ft2 = pd.merge(left=artists_df2,
                        right=audio_features_df2,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft2

,name,song_id,song_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Tropa do Bruxo,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0,0.734,0.228,7.0,-4.731,0.0,0.5300,0.88900,0.00642,0.1020,0.522,162.524,audio_features,00ZKeP47bZtswtANkvxz2j,spotify:track:00ZKeP47bZtswtANkvxz2j,https://api.spotify.com/v1/tracks/00ZKeP47bZts...,https://api.spotify.com/v1/audio-analysis/00ZK...,221538.0,5.0
1,DJ Ws da Igrejinha,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0,0.734,0.228,7.0,-4.731,0.0,0.5300,0.88900,0.00642,0.1020,0.522,162.524,audio_features,00ZKeP47bZtswtANkvxz2j,spotify:track:00ZKeP47bZtswtANkvxz2j,https://api.spotify.com/v1/tracks/00ZKeP47bZts...,https://api.spotify.com/v1/audio-analysis/00ZK...,221538.0,5.0
2,SMU,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0,0.734,0.228,7.0,-4.731,0.0,0.5300,0.88900,0.00642,0.1020,0.522,162.524,audio_features,00ZKeP47bZtswtANkvxz2j,spotify:track:00ZKeP47bZtswtANkvxz2j,https://api.spotify.com/v1/tracks/00ZKeP47bZts...,https://api.spotify.com/v1/audio-analysis/00ZK...,221538.0,5.0
3,Triz,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0,0.734,0.228,7.0,-4.731,0.0,0.5300,0.88900,0.00642,0.1020,0.522,162.524,audio_features,00ZKeP47bZtswtANkvxz2j,spotify:track:00ZKeP47bZtswtANkvxz2j,https://api.spotify.com/v1/tracks/00ZKeP47bZts...,https://api.spotify.com/v1/audio-analysis/00ZK...,221538.0,5.0
4,Mc Menor Thalis,00ZKeP47bZtswtANkvxz2j,Baile do Bruxo,85.0,0.734,0.228,7.0,-4.731,0.0,0.5300,0.88900,0.00642,0.1020,0.522,162.524,audio_features,00ZKeP47bZtswtANkvxz2j,spotify:track:00ZKeP47bZtswtANkvxz2j,https://api.spotify.com/v1/tracks/00ZKeP47bZts...,https://api.spotify.com/v1/audio-analysis/00ZK...,221538.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,Teddy Pendergrass,2nsx3QznZwMFBPdE4PJiJJ,"The More I Get, the More I Want",0.0,0.716,0.808,5.0,-9.491,0.0,0.0662,0.33500,0.00000,0.0950,0.902,121.243,audio_features,2nsx3QznZwMFBPdE4PJiJJ,spotify:track:2nsx3QznZwMFBPdE4PJiJJ,https://api.spotify.com/v1/tracks/2nsx3QznZwMF...,https://api.spotify.com/v1/audio-analysis/2nsx...,264613.0,4.0
1651,Todd Edwards,7xR0hvWymbYc0GHSx4Jrxo,Saved My Life,16.0,0.825,0.911,6.0,-3.918,0.0,0.0522,0.00806,0.78000,0.3270,0.982,124.949,audio_features,7xR0hvWymbYc0GHSx4Jrxo,spotify:track:7xR0hvWymbYc0GHSx4Jrxo,https://api.spotify.com/v1/tracks/7xR0hvWymbYc...,https://api.spotify.com/v1/audio-analysis/7xR0...,415730.0,4.0
1652,Todd Edwards,076XkAjeMf7mfOgGsg3y67,Never Far From You,10.0,0.894,0.917,6.0,-3.810,0.0,0.0904,0.04590,0.87600,0.0585,0.927,124.995,audio_features,076XkAjeMf7mfOgGsg3y67,spotify:track:076XkAjeMf7mfOgGsg3y67,https://api.spotify.com/v1/tracks/076XkAjeMf7m...,https://api.spotify.com/v1/audio-analysis/076X...,400748.0,4.0
1653,Todd Edwards,0i1y9HFHPquPLLIKnxJT23,Dancing For Heaven,6.0,0.866,0.932,6.0,-4.320,1.0,0.0581,0.00851,0.90900,0.0628,0.984,131.993,audio_features,0i1y9HFHPquPLLIKnxJT23,spotify:track:0i1y9HFHPquPLLIKnxJT23,https://api.spotify.com/v1/tracks/0i1y9HFHPquP...,https://api.spotify.com/v1/audio-analysis/0i1y...,349624.0,4.0


In [50]:
df_w_audio_ft2.isna().sum()

name                0
song_id             0
song_name           0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
id                  0
uri                 0
track_href          0
analysis_url        0
duration_ms         0
time_signature      0
dtype: int64

# Next playlist

In [51]:
playlist3 = sp.user_playlist_tracks("spotify", "2LOxEzC4KmoWJ9NhW0kz5M")

In [52]:
playlist3["total"]

992

In [53]:
playlist3["next"]

'https://api.spotify.com/v1/playlists/2LOxEzC4KmoWJ9NhW0kz5M/tracks?offset=100&limit=100&additional_types=track'

In [54]:
all_tracks3 = get_playlist_tracks("2LOxEzC4KmoWJ9NhW0kz5M")
len(all_tracks3)

992

In [55]:
tracks4 = json_normalize(all_tracks3)

In [56]:
tracks4.columns

Index(['added_at', 'is_local', 'primary_color',
       'added_by.external_urls.spotify', 'added_by.href', 'added_by.id',
       'added_by.type', 'added_by.uri', 'track.album.album_type',
       'track.album.artists', 'track.album.available_markets',
       'track.album.external_urls.spotify', 'track.album.href',
       'track.album.id', 'track.album.images', 'track.album.name',
       'track.album.release_date', 'track.album.release_date_precision',
       'track.album.total_tracks', 'track.album.type', 'track.album.uri',
       'track.artists', 'track.available_markets', 'track.disc_number',
       'track.duration_ms', 'track.episode', 'track.explicit',
       'track.external_ids.isrc', 'track.external_urls.spotify', 'track.href',
       'track.id', 'track.is_local', 'track.name', 'track.popularity',
       'track.preview_url', 'track.track', 'track.track_number', 'track.type',
       'track.uri', 'video_thumbnail.url'],
      dtype='object')

In [57]:
len(tracks4)

992

In [58]:
artists_df3 = pd.DataFrame(columns=['track.name'])
for i in tracks4.index:
    artists_for_song2 = json_normalize(tracks4.iloc[i]['track.artists'])
    artists_for_song2['song_id']    = tracks4.iloc[i]['track.id']         # we want to keep song_id, it is the sae for all artists
    artists_for_song2['song_name']  = tracks4.iloc[i]['track.name']       # we want to keep song_name, it is the sae for all artists
    artists_for_song2['popularity'] = tracks4.iloc[i]['track.popularity'] # same for popularity   
    artists_df3 = pd.concat([artists_df3, artists_for_song2], axis=0)

In [59]:
artists_df3.head(20)

,track.name,href,id,name,type,uri,external_urls.spotify,song_id,song_name,popularity
0,NaN,https://api.spotify.com/v1/artists/1dfeR4HaWDb...,1dfeR4HaWDbWqFHLkxsg1d,Queen,artist,spotify:artist:1dfeR4HaWDbWqFHLkxsg1d,https://open.spotify.com/artist/1dfeR4HaWDbWqF...,1AhDOtG9vPSOmsWgNW0BEY,Bohemian Rhapsody - Remastered 2011,0.0
0,NaN,https://api.spotify.com/v1/artists/3qm84nBOXUE...,3qm84nBOXUEQ2vnTfUTTFC,Guns N' Roses,artist,spotify:artist:3qm84nBOXUEQ2vnTfUTTFC,https://open.spotify.com/artist/3qm84nBOXUEQ2v...,7o2CTH4ctstm8TNelqjb51,Sweet Child O' Mine,12.0
0,NaN,https://api.spotify.com/v1/artists/6olE6TJLqED...,6olE6TJLqED3rqDCT0FyPh,Nirvana,artist,spotify:artist:6olE6TJLqED3rqDCT0FyPh,https://open.spotify.com/artist/6olE6TJLqED3rq...,1f3yAtsJtY87CTmM8RLnxf,Smells Like Teen Spirit,4.0
0,NaN,https://api.spotify.com/v1/artists/4x1nvY2FN8j...,4x1nvY2FN8jxqAFA0DA02H,John Lennon,artist,spotify:artist:4x1nvY2FN8jxqAFA0DA02H,https://open.spotify.com/artist/4x1nvY2FN8jxqA...,7pKfPomDEeI4TPT6EOYjn9,Imagine - Remastered 2010,81.0
0,NaN,https://api.spotify.com/v1/artists/1QxaPWG1POM...,1QxaPWG1POM8Ul6WwsHq4y,John Farnham,artist,spotify:artist:1QxaPWG1POM8Ul6WwsHq4y,https://open.spotify.com/artist/1QxaPWG1POM8Ul...,5NwkWwfRJaT55hEPtCmJHx,You're the Voice,63.0
0,NaN,https://api.spotify.com/v1/artists/36QJpDe2go2...,36QJpDe2go2KgaRleHCDTp,Led Zeppelin,artist,spotify:artist:36QJpDe2go2KgaRleHCDTp,https://open.spotify.com/artist/36QJpDe2go2Kga...,5CQ30WqJwcep0pYcV4AMNc,Stairway to Heaven - Remaster,81.0
0,NaN,https://api.spotify.com/v1/artists/5a2EaR3hamo...,5a2EaR3hamoenG9rDuVn8j,Prince,artist,spotify:artist:5a2EaR3hamoenG9rDuVn8j,https://open.spotify.com/artist/5a2EaR3hamoenG...,54X78diSLoUDI3joC2bjMz,Purple Rain,77.0
0,NaN,https://api.spotify.com/v1/artists/711MCceyCBc...,711MCceyCBcFnzjGY4Q7Un,AC/DC,artist,spotify:artist:711MCceyCBcFnzjGY4Q7Un,https://open.spotify.com/artist/711MCceyCBcFnz...,7nemcVsXVFZF01iqpIIo2Y,It's a Long Way to the Top (If You Wanna Rock ...,75.0
0,NaN,https://api.spotify.com/v1/artists/3fMbdgg4jU1...,3fMbdgg4jU18AjLCKBhRSm,Michael Jackson,artist,spotify:artist:3fMbdgg4jU18AjLCKBhRSm,https://open.spotify.com/artist/3fMbdgg4jU18Aj...,20efeySIfZoiSaISGLBbNs,Thriller - Single Version,57.0
0,NaN,https://api.spotify.com/v1/artists/22bE4uQ6baN...,22bE4uQ6baNwSHPVcDxLCe,The Rolling Stones,artist,spotify:artist:22bE4uQ6baNwSHPVcDxLCe,https://open.spotify.com/artist/22bE4uQ6baNwSH...,7fSGbZLhWlAiCC3HDPAULu,(I Can't Get No) Satisfaction - Mono Version /...,0.0


In [60]:
artists_df3 = artists_df3.drop(['href','track.name', 'id', 'type','uri','external_urls.spotify'], axis=1)
artists_df3.head(20)

,name,song_id,song_name,popularity
0,Queen,1AhDOtG9vPSOmsWgNW0BEY,Bohemian Rhapsody - Remastered 2011,0.0
0,Guns N' Roses,7o2CTH4ctstm8TNelqjb51,Sweet Child O' Mine,12.0
0,Nirvana,1f3yAtsJtY87CTmM8RLnxf,Smells Like Teen Spirit,4.0
0,John Lennon,7pKfPomDEeI4TPT6EOYjn9,Imagine - Remastered 2010,81.0
0,John Farnham,5NwkWwfRJaT55hEPtCmJHx,You're the Voice,63.0
0,Led Zeppelin,5CQ30WqJwcep0pYcV4AMNc,Stairway to Heaven - Remaster,81.0
0,Prince,54X78diSLoUDI3joC2bjMz,Purple Rain,77.0
0,AC/DC,7nemcVsXVFZF01iqpIIo2Y,It's a Long Way to the Top (If You Wanna Rock ...,75.0
0,Michael Jackson,20efeySIfZoiSaISGLBbNs,Thriller - Single Version,57.0
0,The Rolling Stones,7fSGbZLhWlAiCC3HDPAULu,(I Can't Get No) Satisfaction - Mono Version /...,0.0


In [65]:
chunks3 = [(i, i+100) for i in range(0, len(artists_df3), 100)]
chunks3
audio_features_list3 = []
for chunk in chunks:
    id_list1003 = artists_df3['song_id'][chunk[0]:chunk[1]]
    audio_features_list3 = audio_features_list3 + sp.audio_features(id_list1003)
    sleep(randint(1,3000)/1000)
len(audio_features_list3)

1176

In [66]:
audio_features_df3 = json_normalize(audio_features_list3)

In [67]:
audio_features_df3.drop_duplicates(inplace=True)

In [68]:
df_w_audio_ft3 = pd.merge(left=artists_df3,
                        right=audio_features_df3,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft3

,name,song_id,song_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Queen,1AhDOtG9vPSOmsWgNW0BEY,Bohemian Rhapsody - Remastered 2011,0.0,0.414,0.404,0.0,-9.928,0.0,0.0499,0.271000,0.000000,0.3000,0.224,71.105,audio_features,1AhDOtG9vPSOmsWgNW0BEY,spotify:track:1AhDOtG9vPSOmsWgNW0BEY,https://api.spotify.com/v1/tracks/1AhDOtG9vPSO...,https://api.spotify.com/v1/audio-analysis/1AhD...,354320.0,4.0
1,Guns N' Roses,7o2CTH4ctstm8TNelqjb51,Sweet Child O' Mine,12.0,0.454,0.910,6.0,-7.766,1.0,0.0448,0.086600,0.099600,0.1160,0.629,125.116,audio_features,7o2CTH4ctstm8TNelqjb51,spotify:track:7o2CTH4ctstm8TNelqjb51,https://api.spotify.com/v1/tracks/7o2CTH4ctstm...,https://api.spotify.com/v1/audio-analysis/7o2C...,354520.0,4.0
2,Nirvana,1f3yAtsJtY87CTmM8RLnxf,Smells Like Teen Spirit,4.0,0.502,0.912,1.0,-4.556,1.0,0.0564,0.000025,0.000173,0.1060,0.720,116.761,audio_features,1f3yAtsJtY87CTmM8RLnxf,spotify:track:1f3yAtsJtY87CTmM8RLnxf,https://api.spotify.com/v1/tracks/1f3yAtsJtY87...,https://api.spotify.com/v1/audio-analysis/1f3y...,301920.0,4.0
3,John Lennon,7pKfPomDEeI4TPT6EOYjn9,Imagine - Remastered 2010,81.0,0.547,0.257,0.0,-12.358,1.0,0.0252,0.907000,0.183000,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867.0,4.0
4,John Farnham,5NwkWwfRJaT55hEPtCmJHx,You're the Voice,63.0,0.369,0.762,10.0,-8.993,1.0,0.0464,0.220000,0.000028,0.5790,0.644,84.957,audio_features,5NwkWwfRJaT55hEPtCmJHx,spotify:track:5NwkWwfRJaT55hEPtCmJHx,https://api.spotify.com/v1/tracks/5NwkWwfRJaT5...,https://api.spotify.com/v1/audio-analysis/5Nwk...,302107.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,Jackson Browne,4MZEZz8MqVgvIMXU6AVP22,Running on Empty,0.0,0.531,0.948,9.0,-5.438,1.0,0.0568,0.006000,0.011700,0.6940,0.498,136.891,audio_features,4MZEZz8MqVgvIMXU6AVP22,spotify:track:4MZEZz8MqVgvIMXU6AVP22,https://api.spotify.com/v1/tracks/4MZEZz8MqVgv...,https://api.spotify.com/v1/audio-analysis/4MZE...,298093.0,4.0
1065,The Police,5pDmunv6G71w7NSjOPXbYw,Every Little Thing She Does Is Magic - Remaste...,0.0,0.535,0.745,2.0,-10.123,1.0,0.0335,0.127000,0.079200,0.1790,0.376,163.913,audio_features,5pDmunv6G71w7NSjOPXbYw,spotify:track:5pDmunv6G71w7NSjOPXbYw,https://api.spotify.com/v1/tracks/5pDmunv6G71w...,https://api.spotify.com/v1/audio-analysis/5pDm...,260573.0,4.0
1066,Ms. Lauryn Hill,2Uu8IiLkLY0UXhCHka4Dlr,Doo Wop (That Thing),0.0,0.535,0.505,2.0,-8.926,0.0,0.2450,0.039300,0.000000,0.0923,0.495,99.935,audio_features,2Uu8IiLkLY0UXhCHka4Dlr,spotify:track:2Uu8IiLkLY0UXhCHka4Dlr,https://api.spotify.com/v1/tracks/2Uu8IiLkLY0U...,https://api.spotify.com/v1/audio-analysis/2Uu8...,320267.0,4.0
1067,Thin Lizzy,43DeSV93pJPT4lCZaWZ6b1,The Boys Are Back In Town,76.0,0.445,0.706,8.0,-9.803,1.0,0.0461,0.234000,0.000252,0.2050,0.768,80.823,audio_features,43DeSV93pJPT4lCZaWZ6b1,spotify:track:43DeSV93pJPT4lCZaWZ6b1,https://api.spotify.com/v1/tracks/43DeSV93pJPT...,https://api.spotify.com/v1/audio-analysis/43De...,266720.0,4.0


In [69]:
df_w_audio_ft3.isna().sum()

name                0
song_id             0
song_name           0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
id                  0
uri                 0
track_href          0
analysis_url        0
duration_ms         0
time_signature      0
dtype: int64

In [70]:
df_w_audio_ft.to_csv('playlist1.csv', index=False)
df_w_audio_ft2.to_csv('playlist2.csv', index=False)
df_w_audio_ft3.to_csv('playlist3.csv', index=False)